In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBasedModelConfig:
    root_dir: Path
    local_data_file: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str  
    params_offset: int
    params_input_shape: list
    

In [6]:
from farm_copilot.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from farm_copilot.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,          
        ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    
    def get_prepare_base_model_config(self) -> PrepareBasedModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBasedModelConfig(
            root_dir=Path(config.root_dir),
            local_data_file=Path(self.config.data_ingestion.local_data_file),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_input_shape=self.params.INPUT_SHAPE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_offset=self.params.OFFSET,
        )

        return prepare_base_model_config




In [8]:
import os
import keras
import tensorflow as tf

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBasedModelConfig) -> None:
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.applications.Xception(
            input_shape=tuple(self.config.params_input_shape),
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, classes: int, freeze_all: bool, freeze_till: int, learning_rate: float, offset: int, input_shape: tuple, image_size: tuple):
        if freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        inputs = keras.Input(shape=input_shape)

        # Pre-trained Xception weights requires that input be scaled
        # from (0, 255) to a range of (-1., +1.), the rescaling layer
        # outputs: `(inputs * scale) + offset`

        scale_layer = tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)
        x = scale_layer(inputs)

        # The base model contains batchnorm layers. We want to keep them in inference mode
        # when we unfreeze the base model for fine-tuning, so we make sure that the
        # base_model is running in inference mode here.
        x = model(x, training=False)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.2)(x)  # Regularize with dropout
        outputs = tf.keras.layers.Dense(classes, activation='softmax')(x)
        full_model = tf.keras.Model(inputs, outputs)

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
     
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=len(os.listdir(Path(self.config.local_data_file))),
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate, 
            offset=self.config.params_offset,
            image_size=tuple(self.config.params_image_size),
            input_shape=tuple(self.config.params_input_shape)
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)


    @staticmethod
    def save_model(path: Path, model: keras.Model):
        keras.models.save_model(model=model, filepath=path)
        
  

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-04-04 14:52:40,996: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-04 14:52:40,999: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-04 14:52:41,000: INFO: common: created directory at: artifacts]
[2024-04-04 14:52:41,001: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-04-04 14:52:47,808: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048) 